In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [2]:
import transformers

We also quickly upload some telemetry - this tells us which examples and software versions are getting used so we know where to prioritize our maintenance efforts. We don't collect (or care about) any personally identifiable information, but if you'd prefer not to be counted, feel free to skip this step or delete this cell entirely.

In [3]:
from transformers.utils import send_example_telemetry

send_example_telemetry("multiple_choice_notebook", framework="pytorch")

# Fine-tuning a model on a multiple choice task

In this notebook, we will see how to fine-tune one of the [ Transformers](https://github.com/huggingface/transformers) model to a multiple choice task, which is the task of selecting the most plausible inputs in a given selection. 


In [4]:
model_checkpoint = "bert-base-uncased"
batch_size = 4

## Loading the dataset

We will use the [🤗 Datasets](https://github.com/huggingface/datasets) library to download the data. This can be easily done with the functions `load_dataset`.  

In [5]:
from datasets import load_dataset, load_metric

`load_dataset` will cache the dataset to avoid downloading it again the next time you run this cell.

In [6]:
datasets = load_dataset("cosmos_qa")

Found cached dataset cosmos_qa (/home/ben/.cache/huggingface/datasets/cosmos_qa/default/0.1.0/3e18538cbfdb2c04189b16642715f0f6da3e97ed5df0aadcec3641245b2cf157)


  0%|          | 0/3 [00:00<?, ?it/s]

Each example has a context, question, and answers. Need to pair together the context and the question.

In [7]:
def show_one(example):
    print(f"Context: {example['context']} {example['question']}")
    print(f"  A - {example['answer0']}")
    print(f"  B - {example['answer1']}")
    print(f"  C - {example['answer2']}")
    print(f"  D - {example['answer3']}")
    print(f"\nGround truth: option {['A', 'B', 'C', 'D'][example['label']]}")

In [8]:
show_one(datasets["train"][0])

Context: Good Old War and person L : I saw both of these bands Wednesday night , and they both blew me away . seriously . Good Old War is acoustic and makes me smile . I really can not help but be happy when I listen to them ; I think it 's the fact that they seemed so happy themselves when they played . In the future , will this person go to see other bands play ?
  A - None of the above choices .
  B - This person likes music and likes to see the show , they will see other bands play .
  C - This person only likes Good Old War and Person L , no other bands .
  D - Other Bands is not on tour and this person can not see them .

Ground truth: option B


## Preprocessing the data

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pretraining this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [9]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

We pass along `use_fast=True` to the call above to use one of the fast tokenizers (backed by Rust) from the 🤗 Tokenizers library. Those fast tokenizers are available for almost all models, but if you got an error with the previous call, remove that argument.

We can them write the function that will preprocess our samples. The tricky part is to put all the possible pairs of questions in two big lists before passing them to the tokenizer, then un-flatten the result so that each example has four input ids, attentions masks, etc.

When calling the `tokenizer`, we use the argument `truncation=True`. This will ensure that an input longer that what the model selected can handle will be truncated to the maximum length accepted by the model.

In [10]:
ending_names = ["answer0", "answer1", "answer2", "answer3"]

def preprocess_function(examples):
    # Repeat each first sentence four times to go with the four possibilities of second sentences.
    question_headers = examples["question"]
    questions = [[context + question] * 4 for context, question in zip(examples["context"], examples["question"])]

    # Grab all second sentences possible for each context.
    question_headers = examples["question"]
    second_sentences = [[f"{examples[end][i]}" for end in ending_names] for i, header in enumerate(question_headers)]
    
    # Flatten everything
    first_sentences = sum(questions, [])
    second_sentences = sum(second_sentences, [])
    
    # Tokenize
    tokenized_examples = tokenizer(first_sentences, second_sentences, truncation=True)
    # Un-flatten
    return {k: [v[i:i+4] for i in range(0, len(v), 4)] for k, v in tokenized_examples.items()}

This seems alright, so we can apply this function on all the examples in our dataset, we just use the `map` method of our `dataset` object we created earlier. This will apply the function on all the elements of all the splits in `dataset`, so our training, validation and testing data will be preprocessed in one single command.

In [11]:
encoded_datasets = datasets.map(preprocess_function, batched=True)

Loading cached processed dataset at /home/ben/.cache/huggingface/datasets/cosmos_qa/default/0.1.0/3e18538cbfdb2c04189b16642715f0f6da3e97ed5df0aadcec3641245b2cf157/cache-4ce8db52a4f55f8b.arrow
Loading cached processed dataset at /home/ben/.cache/huggingface/datasets/cosmos_qa/default/0.1.0/3e18538cbfdb2c04189b16642715f0f6da3e97ed5df0aadcec3641245b2cf157/cache-a097bd1924b30ae4.arrow
Loading cached processed dataset at /home/ben/.cache/huggingface/datasets/cosmos_qa/default/0.1.0/3e18538cbfdb2c04189b16642715f0f6da3e97ed5df0aadcec3641245b2cf157/cache-5030b3849420e07f.arrow


## Fine-tuning the model

Now that our data is ready, we can download the pretrained model and fine-tune it. Since all our task is about mutliple choice, we use the `AutoModelForMultipleChoice` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us.

In [12]:
from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer

model = AutoModelForMultipleChoice.from_pretrained(model_checkpoint)

Some weights of BertForMultipleChoice were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


To instantiate a `Trainer`, we will need to define three more things. The most important is the [`TrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments), which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [13]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-cosmos",
    evaluation_strategy = "epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
)

Here we set the evaluation to be done at the end of each epoch, tweak the learning rate, use the `batch_size` defined at the top of the notebook and customize the number of epochs for training, as well as the weight decay.

Then we need to tell our `Trainer` how to form batches from the pre-processed inputs. We haven't done any padding yet because we will pad each batch to the maximum length inside the batch (instead of doing so with the maximum length of the whole dataset). This will be the job of the *data collator*. A data collator takes a list of examples and converts them to a batch (by, in our case, applying padding). Since there is no data collator in the library that works on our specific problem, we will write one, adapted from the `DataCollatorWithPadding`:

In [14]:
from dataclasses import dataclass
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from typing import Optional, Union
import torch

@dataclass
class DataCollatorForMultipleChoice:
    """
    Data collator that will dynamically pad the inputs for multiple choice received.
    """

    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None

    def __call__(self, features):
        label_name = "label" if "label" in features[0].keys() else "labels"
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]["input_ids"])
        flattened_features = [[{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features]
        flattened_features = sum(flattened_features, [])
        
        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        
        # Un-flatten
        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        # Add back labels
        batch["labels"] = torch.tensor(labels, dtype=torch.int64)
        return batch

When called on a list of examples, it will flatten all the inputs/attentions masks etc. in big lists that it will pass to the `tokenizer.pad` method. This will return a dictionary with big tensors (of shape `(batch_size * 4) x seq_length`) that we then unflatten.

We can check this data collator works on a list of features, we just have to make sure to remove all features that are not inputs accepted by our model (something the `Trainer` will do automatically for us after):

In [15]:
accepted_keys = ["input_ids", "attention_mask", "label"]
features = [{k: v for k, v in encoded_datasets["train"][i].items() if k in accepted_keys} for i in range(10)]
batch = DataCollatorForMultipleChoice(tokenizer)(features)

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


All good!

The last thing to define for our `Trainer` is how to compute the metrics from the predictions. We need to define a function for this, which will just use the `metric` we loaded earlier, the only preprocessing we have to do is to take the argmax of our predicted logits:

In [16]:
import numpy as np

def compute_metrics(eval_predictions):
    predictions, label_ids = eval_predictions
    preds = np.argmax(predictions, axis=1)
    return {"accuracy": (preds == label_ids).astype(np.float32).mean().item()}

Then we just need to pass all of this along with our datasets to the `Trainer`:

In [17]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_datasets["train"],
    eval_dataset=encoded_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=DataCollatorForMultipleChoice(tokenizer),
    compute_metrics=compute_metrics,
)

We can now finetune our model by just calling the `train` method:

In [18]:
trainer.train()

  0%|          | 0/18948 [00:00<?, ?it/s]

{'loss': 1.2974, 'learning_rate': 4.868059953557104e-05, 'epoch': 0.08}
{'loss': 1.2222, 'learning_rate': 4.7361199071142074e-05, 'epoch': 0.16}
{'loss': 1.1882, 'learning_rate': 4.604179860671311e-05, 'epoch': 0.24}
{'loss': 1.1794, 'learning_rate': 4.472239814228415e-05, 'epoch': 0.32}
{'loss': 1.147, 'learning_rate': 4.340299767785518e-05, 'epoch': 0.4}
{'loss': 1.1305, 'learning_rate': 4.2083597213426224e-05, 'epoch': 0.47}
{'loss': 1.1137, 'learning_rate': 4.076419674899726e-05, 'epoch': 0.55}
{'loss': 1.0959, 'learning_rate': 3.944479628456829e-05, 'epoch': 0.63}
{'loss': 1.0859, 'learning_rate': 3.812539582013933e-05, 'epoch': 0.71}
{'loss': 1.0704, 'learning_rate': 3.680599535571037e-05, 'epoch': 0.79}
{'loss': 1.03, 'learning_rate': 3.54865948912814e-05, 'epoch': 0.87}
{'loss': 1.0246, 'learning_rate': 3.416719442685244e-05, 'epoch': 0.95}


  0%|          | 0/747 [00:00<?, ?it/s]

{'eval_loss': 1.0667240619659424, 'eval_accuracy': 0.5715242624282837, 'eval_runtime': 28.1247, 'eval_samples_per_second': 106.134, 'eval_steps_per_second': 26.56, 'epoch': 1.0}
{'loss': 0.8738, 'learning_rate': 3.284779396242348e-05, 'epoch': 1.03}
{'loss': 0.6696, 'learning_rate': 3.152839349799451e-05, 'epoch': 1.11}
{'loss': 0.6598, 'learning_rate': 3.0208993033565547e-05, 'epoch': 1.19}
{'loss': 0.6659, 'learning_rate': 2.8889592569136586e-05, 'epoch': 1.27}
{'loss': 0.6804, 'learning_rate': 2.757019210470762e-05, 'epoch': 1.35}
{'loss': 0.66, 'learning_rate': 2.625079164027866e-05, 'epoch': 1.42}
{'loss': 0.6623, 'learning_rate': 2.4931391175849693e-05, 'epoch': 1.5}
{'loss': 0.6605, 'learning_rate': 2.3611990711420732e-05, 'epoch': 1.58}
{'loss': 0.6621, 'learning_rate': 2.2292590246991768e-05, 'epoch': 1.66}
{'loss': 0.6659, 'learning_rate': 2.0973189782562804e-05, 'epoch': 1.74}
{'loss': 0.6398, 'learning_rate': 1.9653789318133843e-05, 'epoch': 1.82}
{'loss': 0.6363, 'learning

  0%|          | 0/747 [00:00<?, ?it/s]

{'eval_loss': 1.516294002532959, 'eval_accuracy': 0.5926297903060913, 'eval_runtime': 27.8094, 'eval_samples_per_second': 107.338, 'eval_steps_per_second': 26.861, 'epoch': 2.0}
{'loss': 0.3181, 'learning_rate': 1.569558792484695e-05, 'epoch': 2.06}
{'loss': 0.2547, 'learning_rate': 1.4376187460417987e-05, 'epoch': 2.14}
{'loss': 0.2454, 'learning_rate': 1.3056786995989024e-05, 'epoch': 2.22}
{'loss': 0.2431, 'learning_rate': 1.173738653156006e-05, 'epoch': 2.3}
{'loss': 0.2999, 'learning_rate': 1.0417986067131096e-05, 'epoch': 2.37}
{'loss': 0.2418, 'learning_rate': 9.098585602702133e-06, 'epoch': 2.45}
{'loss': 0.2634, 'learning_rate': 7.779185138273169e-06, 'epoch': 2.53}
{'loss': 0.2163, 'learning_rate': 6.459784673844206e-06, 'epoch': 2.61}
{'loss': 0.2843, 'learning_rate': 5.140384209415242e-06, 'epoch': 2.69}
{'loss': 0.2573, 'learning_rate': 3.820983744986278e-06, 'epoch': 2.77}
{'loss': 0.2503, 'learning_rate': 2.501583280557315e-06, 'epoch': 2.85}
{'loss': 0.2341, 'learning_r

  0%|          | 0/747 [00:00<?, ?it/s]

{'eval_loss': 3.127169132232666, 'eval_accuracy': 0.575209379196167, 'eval_runtime': 28.1712, 'eval_samples_per_second': 105.959, 'eval_steps_per_second': 26.516, 'epoch': 3.0}
{'train_runtime': 2361.5879, 'train_samples_per_second': 32.091, 'train_steps_per_second': 8.023, 'train_loss': 0.6769299175962256, 'epoch': 3.0}


TrainOutput(global_step=18948, training_loss=0.6769299175962256, metrics={'train_runtime': 2361.5879, 'train_samples_per_second': 32.091, 'train_steps_per_second': 8.023, 'train_loss': 0.6769299175962256, 'epoch': 3.0})

You can now upload the result of the training to the Hub, just execute this instruction:

In [19]:
trainer.push_to_hub()

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

'https://huggingface.co/Benlitzen43/bert-base-uncased-finetuned-cosmos/tree/main/'

In [20]:
trainer.evaluate()

  0%|          | 0/747 [00:00<?, ?it/s]

{'eval_loss': 3.127169132232666,
 'eval_accuracy': 0.575209379196167,
 'eval_runtime': 28.2943,
 'eval_samples_per_second': 105.498,
 'eval_steps_per_second': 26.401,
 'epoch': 3.0}

In [21]:
trainer.predict(encoded_datasets["train"])

  0%|          | 0/6316 [00:00<?, ?it/s]

PredictionOutput(predictions=array([[ -6.4644947 ,  15.190432  , -13.647244  ,  -9.786214  ],
       [ -0.20564961,  -8.322571  ,  -6.9126034 , -14.93871   ],
       [-14.788638  ,   2.0201185 ,  -8.853438  ,  13.134609  ],
       ...,
       [-14.584502  ,  -7.9205174 ,   7.5685143 , -15.133349  ],
       [-14.584502  ,   4.4133034 ,  -7.92052   , -15.133349  ],
       [  5.9007406 , -14.181762  ,  -8.816415  , -14.884135  ]],
      dtype=float32), label_ids=array([1, 0, 3, ..., 2, 1, 0]), metrics={'test_loss': 0.03762782737612724, 'test_accuracy': 0.9900641441345215, 'test_runtime': 211.5984, 'test_samples_per_second': 119.387, 'test_steps_per_second': 29.849})

In [22]:
trainer.predict(encoded_datasets["validation"])

  0%|          | 0/747 [00:00<?, ?it/s]

PredictionOutput(predictions=array([[ -2.662813  ,  -8.011477  , -13.631744  ,  -7.884886  ],
       [ -3.4888628 , -14.650264  , -10.3486595 ,  -8.170195  ],
       [ 13.366292  , -14.2464285 ,  -7.4868526 , -14.544065  ],
       ...,
       [ -2.797572  ,  -9.793706  ,  -1.5497787 , -11.714148  ],
       [ 12.866762  ,   0.52441245,   1.9660931 ,  -7.78509   ],
       [  7.240388  ,   0.5244213 ,   1.9661115 ,  -7.78509   ]],
      dtype=float32), label_ids=array([1, 0, 0, ..., 0, 0, 0]), metrics={'test_loss': 3.127169132232666, 'test_accuracy': 0.575209379196167, 'test_runtime': 27.7901, 'test_samples_per_second': 107.412, 'test_steps_per_second': 26.88})